<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
Автор материала: аналитик-разработчик в команде Яндекс.Метрики Мария Мансурова. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

#### <center>К [статье](https://habrahabr.ru/company/ods/blog/323730/) на Хабре "Предсказываем будущее с помощью библиотеки Facebook Prophet"

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import os

import pandas as pd
from plotly import __version__

print(__version__)  # need 1.9.0 or greater
import pandas as pd
import requests
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

init_notebook_mode(connected=True)


def plotly_df(df, title=""):
    data = []

    for column in df.columns:
        trace = go.Scatter(x=df.index, y=df[column], mode="lines", name=column)
        data.append(trace)

    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)


%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

## Загрузка и предобработка данных
Данные [соревнования](https://inclass.kaggle.com/c/howpop-habrahabr-favs-lognorm) по прогнозу популярности статьи на Хабрахабре.

In [ ]:
habr_df = pd.read_csv("../../data/howpop_train.csv")

In [ ]:
habr_df["published"] = pd.to_datetime(habr_df.published)
habr_df = habr_df[["published", "url"]]
habr_df = habr_df.drop_duplicates()

In [ ]:
aggr_habr_df = habr_df.groupby("published")[["url"]].count()
aggr_habr_df.columns = ["posts"]

In [ ]:
aggr_habr_df = aggr_habr_df.resample("D").apply(sum)
plotly_df(
    aggr_habr_df.resample("W").apply(sum), title="Опубликованные посты на Хабрахабре"
)

## Построение прогноза Prophet

In [ ]:
# pip install pystan
# pip install fbprophet
from fbprophet import Prophet

In [ ]:
predictions = 30

df = aggr_habr_df.reset_index()
df.columns = ["ds", "y"]
df.tail()

In [ ]:
train_df = df[:-predictions]

In [ ]:
m = Prophet()
m.fit(train_df)

In [ ]:
future = m.make_future_dataframe(periods=30)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast.tail()

In [ ]:
print(", ".join(forecast.columns))

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)

## Оценка качества Prophet

In [ ]:
cmp_df = forecast.set_index("ds")[["yhat", "yhat_lower", "yhat_upper"]].join(
    df.set_index("ds")
)

In [ ]:
import numpy as np

cmp_df["e"] = cmp_df["y"] - cmp_df["yhat"]
cmp_df["p"] = 100 * cmp_df["e"] / cmp_df["y"]
np.mean(abs(cmp_df[-predictions:]["p"])), np.mean(abs(cmp_df[-predictions:]["e"]))

## Прогноз с BoxCox

In [ ]:
def invboxcox(y, lmbda):
    if lmbda == 0:
        return np.exp(y)
    else:
        return np.exp(np.log(lmbda * y + 1) / lmbda)

In [ ]:
train_df2 = train_df.copy().fillna(14)
train_df2 = train_df2.set_index("ds")
train_df2["y"], lmbda_prophet = stats.boxcox(train_df2["y"])

train_df2.reset_index(inplace=True)

m2 = Prophet()
m2.fit(train_df2)
future2 = m2.make_future_dataframe(periods=30)

forecast2 = m2.predict(future2)
forecast2["yhat"] = invboxcox(forecast2.yhat, lmbda_prophet)
forecast2["yhat_lower"] = invboxcox(forecast2.yhat_lower, lmbda_prophet)
forecast2["yhat_upper"] = invboxcox(forecast2.yhat_upper, lmbda_prophet)

cmp_df2 = forecast2.set_index("ds")[["yhat", "yhat_lower", "yhat_upper"]].join(
    df.set_index("ds")
)

cmp_df2["e"] = cmp_df2["y"] - cmp_df2["yhat"]
cmp_df2["p"] = 100 * cmp_df2["e"] / cmp_df2["y"]
np.mean(abs(cmp_df2[-predictions:]["p"])), np.mean(abs(cmp_df2[-predictions:]["e"]))

## Визуализация результатов

In [ ]:
def show_forecast(cmp_df, num_predictions, num_values):
    upper_bound = go.Scatter(
        name="Upper Bound",
        x=cmp_df.tail(num_predictions).index,
        y=cmp_df.tail(num_predictions).yhat_upper,
        mode="lines",
        marker=dict(color="444"),
        line=dict(width=0),
        fillcolor="rgba(68, 68, 68, 0.3)",
        fill="tonexty",
    )

    forecast = go.Scatter(
        name="Prediction",
        x=cmp_df.tail(predictions).index,
        y=cmp_df.tail(predictions).yhat,
        mode="lines",
        line=dict(color="rgb(31, 119, 180)"),
    )

    lower_bound = go.Scatter(
        name="Lower Bound",
        x=cmp_df.tail(num_predictions).index,
        y=cmp_df.tail(num_predictions).yhat_lower,
        marker=dict(color="444"),
        line=dict(width=0),
        mode="lines",
    )

    fact = go.Scatter(
        name="Fact",
        x=cmp_df.tail(num_values).index,
        y=cmp_df.tail(num_values).y,
        marker=dict(color="red"),
        mode="lines",
    )

    # Trace order can be important
    # with continuous error bars
    data = [lower_bound, upper_bound, forecast, fact]

    layout = go.Layout(
        yaxis=dict(title="Посты"),
        title="Опубликованные посты на Хабрахабре",
        showlegend=False,
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
show_forecast(cmp_df, predictions, 200)

##  Сравнение с ARIMA моделью

In [ ]:
train_df = train_df.fillna(14).set_index("ds")

In [ ]:
plt.figure(figsize=(15, 10))
sm.tsa.seasonal_decompose(train_df["y"].values, freq=7).plot()
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(train_df["y"])[1])

In [ ]:
train_df.index = pd.to_datetime(train_df.index)

In [ ]:
train_df["y_box"], lmbda = stats.boxcox([1 if x == 0 else x for x in train_df["y"]])
plt.figure(figsize=(15, 7))
train_df.y.plot()
plt.ylabel(u"Posts on Habr")
print("Оптимальный параметр преобразования Бокса-Кокса: %f" % lmbda)
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(train_df["y"])[1])

In [ ]:
train_df["y_box_diff"] = train_df.y_box - train_df.y_box.shift(7)
plt.figure(figsize=(15, 10))
sm.tsa.seasonal_decompose(train_df.y_box_diff[12:].values, freq=7).plot()
print(
    "Критерий Дики-Фуллера: p=%f"
    % sm.tsa.stattools.adfuller(train_df.y_box_diff[8:])[1]
)

In [ ]:
plt.figure(figsize=(15, 8))
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(train_df.y_box_diff[13:].values.squeeze(), lags=48, ax=ax)
ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(train_df.y_box_diff[13:].values.squeeze(), lags=48, ax=ax)

Начальные приближения Q = 1, q = 4, P = 5, p = 3

In [ ]:
ps = range(0, 4)
d = 1
qs = range(0, 5)
Ps = range(0, 7)
D = 1
Qs = range(0, 2)

In [ ]:
from itertools import product

parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
%%time
results = []
best_aic = float("inf")


for param in parameters_list:
    print(param)
    # try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        %time model = sm.tsa.statespace.SARIMAX(\
                  train_df.y_box,\
                  order=(param[0], d, param[1]),\
                  seasonal_order=(param[2], D, param[3], 7),\
              ).fit(\
                  disp=-1\
              )
    # выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError:
        print("wrong parameters:", param)
        continue
    aic = model.aic
    # сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

warnings.filterwarnings("default")

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ["parameters", "aic"]
print(result_table.sort_values(by="aic", ascending=True).head())

In [ ]:
print(best_model.summary())

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(211)
best_model.resid[13:].plot()
plt.ylabel(u"Residuals")

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=48, ax=ax)

print("Критерий Стьюдента: p=%f" % stats.ttest_1samp(best_model.resid[13:], 0)[1])
print(
    "Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(best_model.resid[13:])[1]
)

In [ ]:
train_df["arima_model"] = invboxcox(best_model.fittedvalues, lmbda)
plt.figure(figsize=(15, 7))
train_df.y.tail(200).plot()
train_df.arima_model[13:].tail(200).plot(color="r")
plt.ylabel("Posts on Habr");

In [ ]:
arima_df = train_df2.set_index("ds")[["y"]]

date_list = [
    pd.datetime.strptime("2016-10-01", "%Y-%m-%d") + pd.Timedelta(x)
    for x in range(0, predictions + 1)
]
future = pd.DataFrame(index=date_list, columns=arima_df.columns)
arima_df = pd.concat([arima_df, future])
arima_df["forecast"] = invboxcox(
    best_model.predict(
        start=train_df.shape[0], end=train_df.shape[0] + predictions - 1
    ),
    lmbda,
)
plt.figure(figsize=(15, 7))
arima_df.y.tail(200).plot()
arima_df.forecast.tail(200).plot(color="r")
plt.ylabel("Habr posts");

In [ ]:
cmp_df.head()

In [ ]:
cmp_df = cmp_df.join(arima_df[["forecast"]])

In [ ]:
import numpy as np

cmp_df["e_arima"] = cmp_df["y"] - cmp_df["forecast"]
cmp_df["p_arima"] = 100 * cmp_df["e_arima"] / cmp_df["y"]

In [ ]:
num_values = 200

forecast = go.Scatter(
    name="Prophet",
    x=cmp_df.tail(predictions).index,
    y=cmp_df.tail(predictions).yhat,
    mode="lines",
    line=dict(color="rgb(31, 119, 180)"),
)


fact = go.Scatter(
    name="Fact",
    x=cmp_df.tail(num_values).index,
    y=cmp_df.tail(num_values).y,
    marker=dict(color="red"),
    mode="lines",
)

arima = go.Scatter(
    name="ARIMA",
    x=cmp_df.tail(predictions).index,
    y=cmp_df.tail(predictions).forecast,
    mode="lines",
)

# Trace order can be important
# with continuous error bars
data = [forecast, fact, arima]

layout = go.Layout(
    yaxis=dict(title="Посты"),
    title="Опубликованные посты на Хабрахабре",
    showlegend=True,
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)